Creating the surrogates for the different pumps

In [1]:
# PRO Feed Pump at 100% speed. Efficiency as a function of flowrate.

import pandas as pd
import numpy as np

path = "C:\\Users\\rchurchi\\flex_desal\\src\wrd\\data\\PRO_Feed_Pump_Data.csv" 
data = pd.read_csv(path)

In [2]:
# Remove the rows with no efficiency value
eff_data = data.dropna(subset=["Efficiency"])

flowrate = eff_data["Flow (gpm)"]
eta = eff_data["Efficiency"]
print(flowrate)
print(eta)
#flowrate["i"] = np.arange(1,9).transpose()
#print(flowrate)
#flowrate = flowrate.set_index("i",inplace=True)


15    1500
19    1875
24    2275
29    2675
31    2778
33    2850
37    3200
39    3400
42    3700
Name: Flow (gpm), dtype: int64
15    0.5900
19    0.6800
24    0.7500
29    0.8000
31    0.8245
33    0.8300
37    0.8300
39    0.8000
42    0.7550
Name: Efficiency, dtype: float64


In [9]:
# Use data to create a basic fit
from idaes.core.surrogate.pysmo_surrogate import (
    PysmoPolyTrainer,
    PysmoSurrogate,
)

input_labels = ["Flow (gpm)"]
output_labels = ["Efficiency"]
input_bounds = {"Flow (gpm)": (1400,3700)}

trainer = PysmoPolyTrainer(
        input_labels=input_labels,
        output_labels=output_labels,
        training_dataframe=eff_data,
     #   max_fraction_training_samples = 1
     #   cross_validation=False
    )

trainer.config.maximum_polynomial_order = 3
#trainer.config.max_fraction_training_samples = 1

# Train Data
trained_surr = trainer.train_surrogate()
# Display

Surrogate = PysmoSurrogate(
    trained_surr,
    input_labels,
    output_labels,
    input_bounds,
)


===========================Polynomial Regression===============================================


No iterations will be run.
Default parameter estimation method is used.
Parameter estimation method:  pyomo 

max_fraction_training_samples set at  0.5
Number of adaptive samples (no_adaptive_samples) set at  4
Maximum number of iterations (Max_iter) set at:  0



Initial surrogate model is of order 3  with a cross-val error of 0.000080
Initial Regression Model Performance:
Order:  3  / MAE: 0.005153  / MSE: 0.000035  / R^2: 0.993940  / Adjusted R^2: 0.990304

Polynomial regression generates a good surrogate model for the input data.

-------------------------------------------------
-------------------------------------------------
Best solution found:  
Order:  3  / MAE: 0.005153  / MSE: 0.000035  / R_sq: 0.993940  / Adjusted R^2: 0.990304

------------------------------------------------------------
The final coefficients of the regression terms are: 

k               | 0.357349
(x_ 1 )^ 1      | 3e-06
(x_ 1 )^ 2      | 0.0
(x_ 1 )^ 3      | -0.0

Results saved in  solution.pickle
2025-12-02 20:48:42 [INFO] idaes.core.surrogate.pysmo_surrogate: Model for output Efficiency trained successfully


NOT ENOUGH DATAPOINTS FOR THIS APPROACH ATM

In [ ]:
# Stuggling with the syntax here
from pyomo.environ import (
    ConcreteModel,
    Objective,
    Expression,
    Var,
    Param,
    Constraint,
    RangeSet,
    minimize
) 

m = ConcreteModel()
m.i = RangeSet(0,2) # Fit Coefficients
m.j = RangeSet(0,8)
a = Var(m.i)
m.flow = Param(m.j,initialize=flowrate)
m.flow = Param(m.j,initialize=eta)

s = sum(((m.a[0]+m.a[1]*m.flow[j]+m.a[2]*m.flow[j]**2) - m.eff[j])**2 for j in m.j)

m.obj = Objective(expr=s, sense = minimize) 



Or just use excel for with a 2nd order fit (w)
scaled vars => sc_Eta = eta*100
sc_flow = flowrate/100

sc_eta = -0.0035sc_flowrate^3 + 0.1647cs_flow^2 + 0.3376sc_Flow + 38.921